In [3]:
import numpy as np
import torch
import torch.nn as nn
import time
import matplotlib.pyplot as plt
import json
import pydde as dde
import os

########################################
#PARAMETERS
nTimeSteps = 60; #seconds
use_case = 'cube-drag'
timestamp = '1814'
input_size = 25
samplenum_test = 5000
samplenum_og = 1000
#criterion = nn.SmoothL1Loss()  # Huber Loss
criterion = nn.MSELoss(reduction= 'sum')  # Huber Loss
if samplenum_og == 15000:
    model_file_path_active = '/Users/nicovonau/Code/thesis_pytorch/Trained_Models/state_dict/Model_statedict_active_' + use_case + f'_{nTimeSteps}tsteps_latest.pt'
    model_file_path_passive = '/Users/nicovonau/Code/thesis_pytorch/Trained_Models/state_dict/Model_statedict_passive_' + use_case + f'_{nTimeSteps}tsteps_latest.pt'
else:
    model_file_path_active = '/Users/nicovonau/Code/thesis_pytorch/Trained_Models/state_dict/Model_statedict_active_' + use_case + f'_{nTimeSteps}tsteps_latest_{samplenum_og}.pt'
    model_file_path_passive = '/Users/nicovonau/Code/thesis_pytorch/Trained_Models/state_dict/Model_statedict_passive_' + use_case + f'_{nTimeSteps}tsteps_latest_{samplenum_og}.pt'
sample_file_path = f'/Users/nicovonau/Code/thesis_pytorch/Data/Samples/data_'+ use_case + f'_{nTimeSteps}tsteps_'+ timestamp +'/'
simulation_file_path = '/Users/nicovonau/Code/thesis_pytorch/Data/Simulations/cube-drag.sim'
objective_file_path = f'/Users/nicovonau/Code/thesis_pytorch/Data/Objectives/cube-drag.obj'
#print(os.listdir(sample_file_path))

# set log level
dde.set_log_level(dde.LogLevel.off)

#######################################
# LOAD SIMULATION AND OBJECTIVE FUNCTION
dyn = dde.DynamicSequence()
dyn.loadFile(simulation_file_path, nTimeSteps)
p_init = np.zeros(dyn.p0.size*nTimeSteps)
for i in range(nTimeSteps):
	p_init[i*dyn.p0.size : (i+1)*dyn.p0.size] = dyn.p0
state_init = dyn.q(p_init)
r = dyn.r(state_init, p_init)
dr = dyn.dr_dp(state_init, p_init)
dq = dyn.dq_dp(state_init, p_init)

#########################################
#LOAD ORIGINAL SAMPLES
og_number_of_files = len(os.listdir(sample_file_path))-5
with open(sample_file_path + f'data_0.json') as json_file:
    data = json.load(json_file)
    filesize = len(data['q'])
og_samplenum = filesize*og_number_of_files
og_input = np.zeros((og_samplenum, input_size))
og_loss = np.zeros(og_samplenum)
og_iterations = np.zeros(og_samplenum)
og_lineSearchIterations = np.zeros(og_samplenum)

for filenum in range(og_number_of_files):
    with open(sample_file_path + f'data_{filenum}.json') as json_file:
        data = json.load(json_file)
        for i, q_t in enumerate(data['q_target']):
            og_input[filenum*filesize+i, 0:3] = np.array(q_t)
        for i, q_i in enumerate(data['q']):
            og_input[filenum*filesize+i, 3:9] = np.array(q_i)
        for i, qdot_i in enumerate(data['qdot']):
            og_input[filenum*filesize+i, 9:15] = np.array(qdot_i)
        for i, qddot_i in enumerate(data['qddot']):
            og_input[filenum*filesize+i, 15:21] = np.array(qddot_i)
        for i, p_now_i in enumerate(data['p_now']):
            og_input[filenum*filesize+i, 21:25] = np.array(p_now_i) - [og_input[filenum*filesize+i, 3],og_input[filenum*filesize+i, 5], og_input[filenum*filesize+i, 6], og_input[filenum*filesize+i, 8]]
        for i, truth in enumerate(data['loss']):
            og_loss[filenum*filesize+i] = np.array(truth)
        for i, iter_ in enumerate(data['iterations']):
            og_iterations[filenum*filesize+i] = np.array(iter_)
        for i, line in enumerate(data['lineSearchIterations']):
            og_lineSearchIterations[filenum*filesize+i] = np.array(line)+1


#########################################
#LOAD TEST SAMPLES
number_of_files = len(os.listdir(sample_file_path + 'data_test/'))
with open(sample_file_path+ 'data_test/' + f'data_0.json') as json_file:
    data = json.load(json_file)
    filesize = len(data['q'])
samplenum = filesize*number_of_files
p = np.zeros((samplenum, dyn.nParameters*nTimeSteps))
input = np.zeros((samplenum, input_size))
loss = np.zeros(samplenum)
iterations = np.zeros(samplenum)
lineSearchIterations = np.zeros(samplenum)

for filenum in range(number_of_files):
    with open(sample_file_path + f'data_{filenum}.json') as json_file:
        data = json.load(json_file)
        for i, p_i in enumerate(data['p']):
            p[filenum*filesize+i, :] = np.array(p_i)
        for i, q_t in enumerate(data['q_target']):
            input[filenum*filesize+i, 0:3] = np.array(q_t)
        for i, q_i in enumerate(data['q']):
            input[filenum*filesize+i, 3:9] = np.array(q_i)
        for i, qdot_i in enumerate(data['qdot']):
            input[filenum*filesize+i, 9:15] = np.array(qdot_i)
        for i, qddot_i in enumerate(data['qddot']):
            input[filenum*filesize+i, 15:21] = np.array(qddot_i)
        for i, p_now_i in enumerate(data['p_now']):
            input[filenum*filesize+i, 21:25] = np.array(p_now_i) - [input[filenum*filesize+i, 3], input[filenum*filesize+i, 5], input[filenum*filesize+i, 6], input[filenum*filesize+i, 8]]
        for i, truth in enumerate(data['loss']):
            loss[filenum*filesize+i] = np.array(truth)
        for i, iter_ in enumerate(data['iterations']):
            iterations[filenum*filesize+i] = np.array(iter_)
        for i, line in enumerate(data['lineSearchIterations']):
            lineSearchIterations[filenum*filesize+i] = np.array(line)+1

        output_size = len(data['p'][0])

print(f'input_size: {input_size}, output_size: {output_size}')
print(f'Shape of og_input: {og_input.shape}')
print(f'Shape of input: {input.shape}')

#Remove zeros
p = p[~(input == 0).all(1)]
og_input = og_input[~(og_input == 0).all(1)]
input = input[~(input == 0).all(1)]

print(f'og_input after removing failed samples: {og_input.shape}')
print(f'input after removing failed samples: {input.shape}')

#normalize qddot
def minmaxscale(input, extrema):
    if extrema == None:
        maximas= []
        minimas= []
        for i in range(len(input[0, :])):
            maximas.append(np.max(input[:,i]))
            minimas.append(np.min(input[:,i]))
        max = np.max(maximas)
        min = np.min(minimas)
        extrema = np.max([max, np.linalg.norm(min)])
        scaled = (input+extrema)/(2*extrema)
        return scaled, extrema
    else:
        scaled = (input+extrema)/(2*extrema)
        return scaled

og_input[:, 15:18], extr_qddot_tran = minmaxscale(og_input[:, 15:18], None)
og_input[:, 18:21], extr_qddot_rot = minmaxscale(og_input[:, 18:21], None)

input[:, 15:18] = minmaxscale(input[:, 15:18], extr_qddot_tran)
input[:, 18:21] = minmaxscale(input[:, 18:21], extr_qddot_rot)

og_input = og_input[0:samplenum_og, :]
input = input[0:samplenum_test, :]
p = p[0:samplenum_test, :]

data = torch.tensor(input).float()
og_data = torch.tensor(og_input).float()

################################
#LOAD MODEL
activeHiddenlayers = [280, 350]
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, activeHiddenlayers[0])
        self.H1 = nn.Linear(activeHiddenlayers[0], activeHiddenlayers[1])
        #self.H1 = nn.Linear(activeHiddenlayers[0], out_sz)
        self.H2 = nn.Linear(activeHiddenlayers[1], out_sz)
        self.L_out = nn.Linear(out_sz, out_sz)
        self.Relu = nn.ReLU(inplace=True)

    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        x = self.H2(x)
        x = self.Relu(x)
        x = self.L_out(x)
        return x

passiveHiddenlayers = [280, 350]
class PassiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(PassiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, passiveHiddenlayers[0])
        self.H1 = nn.Linear(passiveHiddenlayers[0], passiveHiddenlayers[1])
        self.H2 = nn.Linear(passiveHiddenlayers[1], out_sz)
        self.L_out = nn.Linear(out_sz, out_sz)
        self.Relu = nn.ReLU(inplace=True)

    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        x = self.H2(x)
        x = self.Relu(x)
        x = self.L_out(x)
        return x


active_model = ActiveLearn(input_size, output_size)
passive_model = PassiveLearn(input_size, output_size)
active_model.load_state_dict(torch.load(model_file_path_active))
passive_model.load_state_dict(torch.load(model_file_path_passive))
print("MODELS LOADED")

input_size: 25, output_size: 240
Shape of og_input: (21000, 25)
Shape of input: (11000, 25)
og_input after removing failed samples: (20979, 25)
input after removing failed samples: (10994, 25)
MODELS LOADED


In [5]:
############################################
#GENERATE OPTIMIZATION
opt = dde.Newton()

####################################
#SAMPLE STATES
# target (0:3), q (3:9), qdot (9:15), qddot (15:21), p_now (21:25) 
start_time = time.time()
dyn_json = json.load(open(simulation_file_path))
objective_json = json.load(open(objective_file_path))
obj = dde.InverseObjective(dyn)
obj.loadJson(objective_json)

#Trajectory Optimization
p_i = opt.minimize(obj, p_init)
print(f'Time through trajectory optimization: {(time.time() - start_time)}')

start_time = time.time()
with torch.no_grad():
    p_val = passive_model(data[1, :])
print(f'Time through neural network: {(time.time() - start_time)}')

Time through trajectory optimization: 1.3260560035705566
Time through neural network: 0.0011179447174072266
